# Setul de date

Vom analiza un set de date ce conține informații despre persoanele ce au efectuat zboruri cu o anumită companie aeriană cu scopul de a cunoaște nivelul de satisfacție al acestora. Dataset-ul împreună cu detalii suplimentare pot fi găsite [aici](https://www.kaggle.com/datasets/teejmahal20/airline-passenger-satisfaction).

<img src="https://cdn-icons-png.flaticon.com/512/5566/5566691.png" alt="Airplane"/>

Fișierul .csv conține următoarele coloane relevante:


*   **Gender**: genul persoanei (**Male** sau **Female**)
*   **Customer Type**: tipul clientului din punct de vedere al frecvenței utilizării companiei (**Loyal Customer** sau **dysloyal Customer**)
*   **Age**: vârsta pasagerilor
*   **Type of Travel**: scopul călătoriei (**Personal Travel** sau **Business travel**)
*   **Class**: clasa de zbor (**Eco**, **Eco Plus** sau **Business**)
*   **Flight distance**: distanța zborului
*   **Inflight wifi service**: nivelul de satisfacție cu serviciul wifi la bord (**0** - N/A, **1-5**)
*   **Departure/Arrival time convenient**: nivelul de satisfacție cu orele la care au fost programate decolare/aterizarea
*   **Ease of Online booking**: nivelul de satisfacție cu procesul de rezervare online a biletelor
*   **Gate location**: nivelul de satisfacție cu locația porții de îmbarcare
*   **Food and drink**: nivelul de satisfacție cu mâncarea și băutura de la bord
*   **Online boarding**: nivelul de satisfacție legat de procesul îmbarcării online
*   **Seat comfort**: nivelul de satisfacție legat de confortul locului din avion
*   **Inflight entertainment**: nivelul de satisfacție cu mijloacele de divertisment de la bord
*   **On-board service**: nivelul de satisfacție cu serviciile de la bord
*   **Leg room service**: nivelul de satisfacție cu spațiul locului din avion
*   **Baggage handling**: nivelul de satisfacție cu procesul de lăsare/colectare a bagajelor
*   **Check-in service**: nivelul de satisfacție cu procesul de Check In
*   **Inflight service**: nivelul de satisfacție legat de serviciile însoțitorilor de zbor
*   **Cleanliness**: nivelul de satisfacție legat de curățenie
*   **Departure Delay in Minutes**: numărul de minute de întârziere pe care le-a avut aeronava la plecare
*   **Arrival Delay in Minutes**: numărul de minute de întârziere pe care le-a avut aeronava la aterizare
*   **Satisfaction**: nivelul general de satisfacție legat de companie (**satisfied** sau **neutral or dissatisfied**)

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("airline").getOrCreate()

In [ ]:
airline = spark.read.csv("/content/drive/MyDrive/Proiect Big Data/airline_satisfaction.csv", header=True, inferSchema=True)

airline.show()

+---+------+------+-----------------+---+---------------+--------+---------------+---------------------+---------------------------------+----------------------+-------------+--------------+---------------+------------+----------------------+----------------+----------------+----------------+---------------+----------------+-----------+--------------------------+------------------------+--------------------+
|_c0|    id|Gender|    Customer Type|Age| Type of Travel|   Class|Flight Distance|Inflight wifi service|Departure/Arrival time convenient|Ease of Online booking|Gate location|Food and drink|Online boarding|Seat comfort|Inflight entertainment|On-board service|Leg room service|Baggage handling|Checkin service|Inflight service|Cleanliness|Departure Delay in Minutes|Arrival Delay in Minutes|        satisfaction|
+---+------+------+-----------------+---+---------------+--------+---------------+---------------------+---------------------------------+----------------------+-------------+-

# Curățarea și transformarea datelor

Deoarece primele două coloane (numărul curent și id-ul) nu ne sunt de folos, putem renunța la ele.

De asmenea, observăm o inconsistență între anumite clase cu privire la scrierea cu majuscule sau minuscule (Personal **T**ravel vs Business **t**ravel, **L**oyal Customer vs **d**isloyal Customer) pe care le vom remedia.

In [ ]:
from pyspark.sql.functions import initcap, substring, translate

airline_clean = airline.drop("_c0", "id")
airline_clean = airline_clean.withColumn("Customer Type", initcap(airline_clean["Customer Type"]))
airline_clean = airline_clean.withColumn("Type of Travel", translate("Type of Travel", 't', 'T'))

airline_clean.show()

+------+-----------------+---+---------------+--------+---------------+---------------------+---------------------------------+----------------------+-------------+--------------+---------------+------------+----------------------+----------------+----------------+----------------+---------------+----------------+-----------+--------------------------+------------------------+--------------------+
|Gender|    Customer Type|Age| Type of Travel|   Class|Flight Distance|Inflight wifi service|Departure/Arrival time convenient|Ease of Online booking|Gate location|Food and drink|Online boarding|Seat comfort|Inflight entertainment|On-board service|Leg room service|Baggage handling|Checkin service|Inflight service|Cleanliness|Departure Delay in Minutes|Arrival Delay in Minutes|        satisfaction|
+------+-----------------+---+---------------+--------+---------------+---------------------+---------------------------------+----------------------+-------------+--------------+---------------+---

In [ ]:
# calculăm numărul de valori null din fiecare coloană
from pyspark.sql.functions import col, isnan, when, count

airline_clean.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in airline_clean.columns]).show()
print("Dataset contains:", airline_clean.count(), "rows.")

+------+-------------+---+--------------+-----+---------------+---------------------+---------------------------------+----------------------+-------------+--------------+---------------+------------+----------------------+----------------+----------------+----------------+---------------+----------------+-----------+--------------------------+------------------------+------------+
|Gender|Customer Type|Age|Type of Travel|Class|Flight Distance|Inflight wifi service|Departure/Arrival time convenient|Ease of Online booking|Gate location|Food and drink|Online boarding|Seat comfort|Inflight entertainment|On-board service|Leg room service|Baggage handling|Checkin service|Inflight service|Cleanliness|Departure Delay in Minutes|Arrival Delay in Minutes|satisfaction|
+------+-------------+---+--------------+-----+---------------+---------------------+---------------------------------+----------------------+-------------+--------------+---------------+------------+----------------------+-------

Observăm că singura coloană ce conține valori null este **Arrival Delay in Minutes** (310). Deoarece acest număr este foarte mic față de numărul total de linii din setul de date (103904) alegem să ștergem intrările ce conțin valori null.

In [ ]:
airline_data = airline_clean.na.drop()

airline_data.describe().show()

+-------+------+-----------------+------------------+---------------+--------+------------------+---------------------+---------------------------------+----------------------+------------------+------------------+------------------+------------------+----------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+--------------------------+------------------------+--------------------+
|summary|Gender|    Customer Type|               Age| Type of Travel|   Class|   Flight Distance|Inflight wifi service|Departure/Arrival time convenient|Ease of Online booking|     Gate location|    Food and drink|   Online boarding|      Seat comfort|Inflight entertainment|  On-board service|  Leg room service|  Baggage handling|  Checkin service| Inflight service|       Cleanliness|Departure Delay in Minutes|Arrival Delay in Minutes|        satisfaction|
+-------+------+-----------------+------------------+---------------+--------+

In [ ]:
# afișăm schema
airline_data.printSchema()

root
 |-- Gender: string (nullable = true)
 |-- Customer Type: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Type of Travel: string (nullable = true)
 |-- Class: string (nullable = true)
 |-- Flight Distance: integer (nullable = true)
 |-- Inflight wifi service: integer (nullable = true)
 |-- Departure/Arrival time convenient: integer (nullable = true)
 |-- Ease of Online booking: integer (nullable = true)
 |-- Gate location: integer (nullable = true)
 |-- Food and drink: integer (nullable = true)
 |-- Online boarding: integer (nullable = true)
 |-- Seat comfort: integer (nullable = true)
 |-- Inflight entertainment: integer (nullable = true)
 |-- On-board service: integer (nullable = true)
 |-- Leg room service: integer (nullable = true)
 |-- Baggage handling: integer (nullable = true)
 |-- Checkin service: integer (nullable = true)
 |-- Inflight service: integer (nullable = true)
 |-- Cleanliness: integer (nullable = true)
 |-- Departure Delay in Minutes: integer 

# Analiza datelor

Pentru fiecare clasă și nivel general de satisfacție calculăm numărul de adulți ce au călătorit în scop profesional.

In [ ]:
airline_data.filter((airline_data["Age"] >= 18) & (airline_data["Type of Travel"] == "Business Travel")) \
            .groupBy(["Class", "satisfaction"]) \
            .count() \
            .orderBy("Class", ascendng = [False]) \
            .show(truncate=False)

+--------+-----------------------+-----+
|Class   |satisfaction           |count|
+--------+-----------------------+-----+
|Business|neutral or dissatisfied|12787|
|Business|satisfied              |33663|
|Eco     |neutral or dissatisfied|13306|
|Eco     |satisfied              |5803 |
|Eco Plus|neutral or dissatisfied|2199 |
|Eco Plus|satisfied              |1473 |
+--------+-----------------------+-----+



Vrem să afișăm pentru fiecare gen și clasă: vârsta mediană, distanța minimă și maximă parcursă de persoanele satisfăcute de companie și ale căror întârzieri nu depășesc media.

In [ ]:
airline_data.createOrReplaceTempView("airline")

In [ ]:
spark.sql("""SELECT Gender, Class, MEDIAN(Age) AS median_age,
                    MIN(`Flight Distance`) AS min_distance, MAX(`Flight Distance`) AS max_distance
             FROM airline
             WHERE satisfaction = 'satisfied'
             AND `Arrival Delay in Minutes` <= (SELECT AVG(`Arrival Delay in Minutes`) FROM airline)
             AND `Departure Delay in Minutes` <= (SELECT AVG(`Departure Delay in Minutes`) FROM airline)
             GROUP BY Gender, Class
             ORDER BY 1, 2, 5 DESC""").show()

+------+--------+----------+------------+------------+
|Gender|   Class|median_age|min_distance|max_distance|
+------+--------+----------+------------+------------+
|Female|Business|      44.0|          56|        4983|
|Female|     Eco|      39.0|          31|        4963|
|Female|Eco Plus|      38.0|          67|        2917|
|  Male|Business|      44.0|          67|        4983|
|  Male|     Eco|      39.0|          56|        4963|
|  Male|Eco Plus|      39.0|          31|        4983|
+------+--------+----------+------------+------------+



Pentru fiecare tip de client, clasă și gen calculăm media, minimul și maximul mediei satisfacției față de serviciile de la bord (acolo unde există evaluări) pentru grupurile care conțin minim 4000 de persoane.

Pentru a afla media notelor pe care le-au acordat pasagerii anumitor servicii vom crea o funcție definită de utilizator ce primește ca parametru un număr arbitrar de valorii de tip *nivel de satisfacție* și returnează media celor diferite de 0 (valoare ce semnifică absența unei note).

In [ ]:
def avg_satisfaction(*args):
  sat_sum = 0
  sat_nr = 0
  for sat in args:
    if sat > 0:
      sat_sum += sat
      sat_nr += 1
  return sat_sum / sat_nr

In [ ]:
# înregistrăm funcția definită anterior ca udf în Spark
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

avg_satisfaction_udf = udf(lambda *z : avg_satisfaction(*z), DoubleType())

In [ ]:
airline_data_avg_sat = airline_data.withColumn("Average on board satisfaction", \
                                               avg_satisfaction_udf(col("Inflight wifi service"), \
                                                                    col("Food and drink"), \
                                                                    col("Seat comfort"), \
                                                                    col("Inflight entertainment"), \
                                                                    col("On-board service"), \
                                                                    col("Leg room service"), \
                                                                    col("Inflight service"), \
                                                                    col("Cleanliness")))

# adăugăm o coloană separată ce reține media notelor serviciilor de la bord
airline_data_avg_sat.show(5)

+------+-----------------+---+---------------+--------+---------------+---------------------+---------------------------------+----------------------+-------------+--------------+---------------+------------+----------------------+----------------+----------------+----------------+---------------+----------------+-----------+--------------------------+------------------------+--------------------+-----------------------------+
|Gender|    Customer Type|Age| Type of Travel|   Class|Flight Distance|Inflight wifi service|Departure/Arrival time convenient|Ease of Online booking|Gate location|Food and drink|Online boarding|Seat comfort|Inflight entertainment|On-board service|Leg room service|Baggage handling|Checkin service|Inflight service|Cleanliness|Departure Delay in Minutes|Arrival Delay in Minutes|        satisfaction|Average on board satisfaction|
+------+-----------------+---+---------------+--------+---------------+---------------------+---------------------------------+-----------

In [ ]:
from pyspark.sql.functions import min, max, avg

airline_data_avg_sat.groupBy(["Customer Type", "Class", "Gender"]) \
                    .agg(min("Average on board satisfaction").alias("min_sat"), \
                         avg("Average on board satisfaction").alias("avg_sat"), \
                         max("Average on board satisfaction").alias("max_sat"), \
                         count('*').alias("count")) \
                    .filter("count > 4000") \
                    .drop("count") \
                    .show()

+-----------------+--------+------+-------+------------------+-------+
|    Customer Type|   Class|Gender|min_sat|           avg_sat|max_sat|
+-----------------+--------+------+-------+------------------+-------+
|   Loyal Customer|     Eco|Female|  1.125|3.1217449942940165|    5.0|
|Disloyal Customer|     Eco|  Male|   1.25| 3.031726595076255|    5.0|
|   Loyal Customer|Business|Female|    1.0| 3.569775085175314|    5.0|
|   Loyal Customer|Business|  Male|    1.0| 3.573174505508376|    5.0|
|Disloyal Customer|     Eco|Female|  1.125| 3.012797365754813|    5.0|
|   Loyal Customer|     Eco|  Male|  1.125| 3.148047057722893|    5.0|
+-----------------+--------+------+-------+------------------+-------+



Pentru fiecare nivel de întârzieri (stabilit cu ajutorul unei funcții definite de utilizator ce calculează suma dintre întârzierile de la decolare și aterizare și asignează una din clasele *No Delay, Small, Average, Long*) afișăm media distanțelor pentru tipul cel mai frecvent de pasageri din grupul respectiv.

In [ ]:
def delay(departure, arrival):
  sum_minutes = departure + arrival
  if sum_minutes == 0:
    return "No Delay"
  elif sum_minutes > 0 and sum_minutes <= 10:
    return "Small"
  elif sum_minutes > 10 and sum_minutes <= 60:
    return "Average"
  elif sum_minutes > 60:
    return "Long"

In [ ]:
# înregistrăm funcția în sesiune pentru a o pute folosi în cereri SQL
from pyspark.sql.types import StringType

spark.udf.register("delay_udf", delay, StringType())

<function __main__.delay(departure, arrival)>

In [ ]:
spark.sql("""WITH aux AS (SELECT delay_udf(`Departure Delay in Minutes`, `Arrival Delay in Minutes`) AS Delay,
                                 `Customer Type`, ROUND(AVG(`Flight Distance`), 2) AS AvgDist, COUNT(*) AS c
                          FROM airline
                          GROUP BY delay_udf(`Departure Delay in Minutes`, `Arrival Delay in Minutes`), `Customer Type`)
             SELECT Delay, `Customer Type`, AvgDist
             FROM aux a
             WHERE c = (SELECT MAX(c)
                        FROM aux
                        WHERE delay = a.delay)""").show()

+--------+--------------+-------+
|   Delay| Customer Type|AvgDist|
+--------+--------------+-------+
|No Delay|Loyal Customer|1276.39|
| Average|Loyal Customer|1321.95|
|   Small|Loyal Customer|1326.44|
|    Long|Loyal Customer|1275.85|
+--------+--------------+-------+

